In [3]:
import os
import json
import time
import csv
import requests 
import urllib
import xml.etree.ElementTree as ET

# get regionId
# http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=X1-ZWz17fxr6iqknf_8xo7s&state=ma&city=westford&childtype=neighborhood
# query property details
# http://www.zillow.com/webservice/GetSearchResults.htm?zws-id=X1-ZWz17fxr6iqknf_8xo7s&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA
# example of region search
# example: https://www.zillow.com/boston-ma/?searchQueryState={%22pagination%22:{},%22mapBounds%22:{%22west%22:-71.1642322561238,%22east%22:-70.98769569187425,%22south%22:42.30783502149961,%22north%22:42.40731071206567},%22regionSelection%22:[{%22regionId%22:44269,%22regionType%22:6}],%22isMapVisible%22:true,%22mapZoom%22:13,%22filterState%22:{},%22isListVisible%22:true}
# https://www.zillow.com/boston-ma/?searchQueryState={%22mapBounds%22:{%22west%22:-71.1642322561238,%22east%22:-70.98769569187425,%22south%22:42.30783502149961,%22north%22:42.40731071206567},%22regionSelection%22:[{%22regionId%22:44269,%22regionType%22:6}],%22filterState%22:{}}
# https://www.zillow.com/boston-ma/?searchQueryState={%22pagination%22:{},%22mapBounds%22:{%22west%22:-71.25250053824857,%22east%22:-70.89942740974948,%22south%22:42.258038131589316,%22north%22:42.456989500076766},%22regionSelection%22:[{%22regionId%22:44269,%22regionType%22:6}],%22isMapVisible%22:true,%22mapZoom%22:12,%22filterState%22:{%22price%22:{%22min%22:500000,%22max%22:1000000},%22monthlyPayment%22:{%22min%22:1853,%22max%22:3706},%22beds%22:{%22min%22:2},%22baths%22:{%22min%22:2},%22hoa%22:{%22max%22:600},%22parkingSpots%22:{%22min%22:1},%22sqft%22:{%22min%22:500,%22max%22:1500}},%22isListVisible%22:true}
# https://www.zillow.com/boston-ma/?searchQueryState={%22pagination%22:{},%22mapBounds%22:{%22west%22:-71.42903710249811,%22east%22:-70.72289084549993,%22south%22:42.16830201931734,%22north%22:42.546314603942044},%22mapZoom%22:11,%22regionSelection%22:[{%22regionId%22:44269,%22regionType%22:6}],%22isMapVisible%22:true,%22filterState%22:{%22price%22:{%22min%22:500000,%22max%22:1000000},%22beds%22:{%22min%22:2},%22baths%22:{%22min%22:2},%22hoa%22:{%22max%22:600},%22parkingSpots%22:{%22min%22:1},%22sqft%22:{%22min%22:500,%22max%22:1500},%22isOpenHousesOnly%22:{%22value%22:true}},%22isListVisible%22:true}

zwsid="X1-ZWz17fxr6iqknf_8xo7s"
region="boston"
state="ma"

searchname="test8"
filepath=os.path.join( "C:\\","Users","James","Documents", "My Docs", "James","test")
filename=os.path.join( filepath, searchname+".html")
csvfile=os.path.join( filepath, searchname+".csv")
jsonfile=os.path.join( filepath, searchname+".json")
xmlfile=os.path.join( filepath, searchname+".xml")

# get the region id of the area of interest
apiurl="http://www.zillow.com/webservice/GetRegionChildren.htm"
apidata={'zws-id':zwsid, 'state':state, 'city':region, 'childtype':"neighborhood"}
full_apiurl=apiurl+"?"+urllib.parse.urlencode(apidata)
#print(full_apiurl)
api_response = requests.get(full_apiurl)
file = open(xmlfile, "w", encoding='utf-8')
response_content=api_response.content.decode('utf-8')
file.write(response_content)
file.close()
# parse the xml file
root = ET.parse(xmlfile).getroot()
print(root)
for type_tag in root.findall('{http://www.zillow.com/static/xsd/RegionChildren.xsd}regionchildren/response/region'):
    value = type_tag.get('id')
    print(value)
    
#tree = ET.fromstring(response_content)
#print(tree)
#lst = tree.findall('RegionChildren:regionchildren/response/region')
#print(lst)
#for item in lst:
#    print item.get('id')

url = "https://www.zillow.com/boston-ma/"
headers = {'User-Agent': 'Chrome/39.0.2171.95'}
#filter='{"pagination":{},"mapBounds":{"west":-71.1642322561238,"east":-70.98769569187425,"south":42.30783502149961,"north":42.40731071206567},"regionSelection":[{"regionId":44269,"regionType":6}],"isMapVisible":true,"mapZoom":13,"filterState":{},"isListVisible":true}'

filter={
    'pagination':{"currentPage":2},
    'mapBounds':{'west':-71.1642322561238,'east':-70.98769569187425,'south':42.30783502149961,'north':42.40731071206567},
    'regionSelection':[{'regionId':44269,'regionType':6}],
    'isMapVisible':True,
    'mapZoom':13,
    'filterState':{},
    'isListVisible':True}

data={'searchQueryState':filter}
url_values = urllib.parse.urlencode(data)
full_url = url + '?' + url_values
#print(full_url)

response = requests.get(full_url, headers=headers)
#print(response.content)

#response =  urllib.request.urlopen(full_url)
file = open(filename, "w", encoding='utf-8')
file.write(str(response.content))
file.close()

try:
    file = open(filename, "r", encoding='utf-8')
except:
    print("File " + filename + " not found")
with file:
    text = file.read()
    while True:
        startPos=text.find("<!--")
        if startPos != -1:
            endPos=text.find("-->")
            if endPos != -1:
                extractedString = text[startPos+4:endPos]
                if 'queryState' in extractedString:
                    extractedString=extractedString.replace("\\", '')
                    #print(extractedString)
                    
                    data = json.loads(extractedString)
                    result = {}
                    result['region']=data['queryState']['regionSelection'][0]['regionId']
                    result['west']=data['queryState']['mapBounds']['west']
                    result['east']=data['queryState']['mapBounds']['east']
                    result['south']=data['queryState']['mapBounds']['south']
                    result['north']=data['queryState']['mapBounds']['north']
                    if 'price' in data['queryState']['filterState']:
                        if 'min' in data['queryState']['filterState']['price']:
                            result['price-min']=data['queryState']['filterState']['price']['min']
                        if 'max' in data['queryState']['filterState']['price']:
                            result['price-max']=data['queryState']['filterState']['price']['max']
                    found_items=data['searchResults']['listResults']
                    result['house-found']=[]
                    for found_item in found_items:
                        # print(found_item)
                        item={}
                        item['address']=""
                        item['zipcode']=""
                        item['city']=""
                        item['state']=""
                        item['latitude']=""
                        item['longitude']=""
                        item['bathrooms']=""
                        item['bedrooms']=""
                        item['livingArea']=""
                        item['yearBuilt']=""
                        item['lotSize']=""
                        item['homeType']=""
                        item['imageLink']=""                        
                        item['priceReduction']=""
                        item['isPreforeclosureAuction']=""
                        item['brokerName']=""
                        item['brokerPhone']=""
                        item['openHouseDescription']=""
                        if 'address' in found_item:
                            item['address']=found_item['address']
                        if 'price' in found_item:
                            item['price']=found_item['price']
                        if 'pricePerSqft' in found_item:
                            item['pricePerSqft']=found_item['pricePerSqft']
                            
                            
                        if 'hdpData' in found_item:
                            if 'homeInfo' in found_item['hdpData']:
                                if 'price' in found_item['hdpData']['homeInfo']:
                                    item['price']=found_item['hdpData']['homeInfo']['price']
                                if 'zipcode' in found_item['hdpData']['homeInfo']:                            
                                    item['zipcode']=found_item['hdpData']['homeInfo']['zipcode']
                                if 'city' in found_item['hdpData']['homeInfo']:                            
                                    item['city']=found_item['hdpData']['homeInfo']['city']
                                if 'state' in found_item['hdpData']['homeInfo']:                            
                                    item['state']=found_item['hdpData']['homeInfo']['state']
                                if 'latitude' in found_item['hdpData']['homeInfo']:                            
                                    item['latitude']=found_item['hdpData']['homeInfo']['latitude']
                                if 'longitude' in found_item['hdpData']['homeInfo']:                            
                                    item['longitude']=found_item['hdpData']['homeInfo']['longitude']
                                if 'bathrooms' in found_item['hdpData']['homeInfo']:                            
                                    item['bathrooms']=found_item['hdpData']['homeInfo']['bathrooms']
                                if 'bedrooms' in found_item['hdpData']['homeInfo']:                            
                                    item['bedrooms']=found_item['hdpData']['homeInfo']['bedrooms']
                                if 'livingArea' in found_item['hdpData']['homeInfo']:                            
                                    item['livingArea']=found_item['hdpData']['homeInfo']['livingArea']
                                if 'yearBuilt' in found_item['hdpData']['homeInfo']:                            
                                    item['yearBuilt']=found_item['hdpData']['homeInfo']['yearBuilt']
                                if 'lotSize' in found_item['hdpData']['homeInfo']:                            
                                    item['lotSize']=found_item['hdpData']['homeInfo']['lotSize']
                                if 'homeType' in found_item['hdpData']['homeInfo']:                            
                                    item['homeType']=found_item['hdpData']['homeInfo']['homeType']
                                if 'imageLink' in found_item['hdpData']['homeInfo']:                            
                                    item['imageLink']=found_item['hdpData']['homeInfo']['imageLink']
                                if 'contactPhone' in found_item['hdpData']['homeInfo']:
                                    item['contactPhone']=found_item['hdpData']['homeInfo']['contactPhone']
                                if 'openHouse' in found_item['hdpData']['homeInfo']:
                                    item['openHouse']=found_item['hdpData']['homeInfo']['openHouse']
                                if 'open_house_info' in found_item['hdpData']['homeInfo']:
                                    if 'open_house_showing' in found_item['hdpData']['homeInfo']['open_house_info']:
                                        open_hours=found_item['hdpData']['homeInfo']['open_house_info']['open_house_showing']
                                        index=0;
                                        for open_hour in open_hours:
                                            start_time_key="open_house_start_"+str(index)
                                            end_time_key="open_house_end_"+str(index)
                                            item[start_time_key] = time.ctime(open_hour['open_house_start']/1000-10800)
                                            item[end_time_key] = time.ctime(open_hour['open_house_end']/1000-10800)
                                            index+=1
                                #item['openHouse']=[]
                                #if 'open_house_info' in found_item['hdpData']['homeInfo']:
                                #    if 'open_house_showing' in found_item['hdpData']['homeInfo']['open_house_info']:
                                #        open_hours=found_item['hdpData']['homeInfo']['open_house_info']['open_house_showing']
                                #        for open_hour in open_hours:
                                #            hour={}
                                #            hour['open_house_start']=time.ctime(open_hour['open_house_start']/1000-10800)
                                #            hour['open_house_end']=time.ctime(open_hour['open_house_end']/1000-10800)                               
                                #            item['openHouse'].append(hour)
                                if 'priceReduction' in found_item['hdpData']['homeInfo']:                            
                                    item['priceReduction']=found_item['hdpData']['homeInfo']['priceReduction']
                                if 'isPreforeclosureAuction' in found_item['hdpData']['homeInfo']:                            
                                    item['isPreforeclosureAuction']=found_item['hdpData']['homeInfo']['isPreforeclosureAuction']
                        if 'brokerName' in found_item:
                            item['brokerName']=found_item['brokerName']
                        if 'brokerPhone' in found_item:
                            item['brokerPhone']=found_item['brokerPhone']
                        if 'openHouseDescription' in found_item:
                            item['openHouseDescription']=found_item['openHouseDescription']
                        result['house-found'].append(item)
                    with open(jsonfile, 'w') as outfile:
                        json.dump(result, outfile)
                    found_list = open(csvfile, 'w')
                    csvwriter = csv.writer(found_list)
                    count = 0
                    for house in result['house-found']:
                        if count == 0:
                            header = house.keys()
                            csvwriter.writerow(header)
                            count += 1
                        csvwriter.writerow(house.values())
                    found_list.close()
                pos = endPos+3
                if pos >= len(text):
                    break
                else:
                    text = text[pos:]
            else:
                break
        else:
            break
            


<Element '{http://www.zillow.com/static/xsd/RegionChildren.xsd}regionchildren' at 0x000002581FEFAD18>
